In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader

In [3]:
import sys
import os

module_path = os.path.abspath("/dcai/users/chache/smrt-foundation")

if module_path not in sys.path:
    sys.path.append(module_path)
    
import copy
import torch
from tqdm import tqdm
from smrt_foundation.probe import SingleIdxProbe
from smrt_foundation.model import Smrt2Vec, SmrtEncoder
from smrt_foundation.dataset import LabeledMemmapDataset


device=torch.device('cuda')

In [17]:
ds = LabeledMemmapDataset('../data/01_processed/val_sets/cpg_neg.memmap/', '../data/01_processed/val_sets/cpg_pos.memmap/')
dl = DataLoader(ds,
                batch_size=256,
                drop_last=True,
                persistent_workers=False,
                prefetch_factor=None,
                shuffle=True)

In [5]:
len(ds)

23183496

In [12]:
ds[0][0]

tensor([[ 1.0000,  1.2637,  0.3120,  0.0000],
        [ 0.0000,  0.2386, -0.7773,  0.0000],
        [ 1.0000, -0.7891, -1.2842,  0.0000],
        [ 1.0000, -0.3613, -0.3579,  0.0000],
        [ 0.0000, -0.9092,  0.1085,  0.0000],
        [ 2.0000,  0.1617, -1.9248,  0.0000],
        [ 3.0000, -0.5654, -1.9248,  0.0000],
        [ 0.0000, -0.4612,  0.4080,  0.0000],
        [ 2.0000, -0.3613, -0.7773,  0.0000],
        [ 3.0000,  1.7471,  0.2124,  0.0000],
        [ 3.0000,  0.4553,  0.7573,  0.0000],
        [ 3.0000, -0.0853,  0.2124,  0.0000],
        [ 3.0000, -0.7891, -0.3579,  0.0000],
        [ 2.0000, -1.7988,  2.9941,  0.0000],
        [ 2.0000, -0.4612,  1.4600,  0.0000],
        [ 3.0000, -0.9092, -0.1136,  0.0000],
        [ 1.0000, -1.7988,  2.6816,  0.0000],
        [ 0.0000, -0.6743,  0.3120,  0.0000],
        [ 2.0000,  0.2386, -1.4795,  0.0000],
        [ 2.0000,  3.0078,  0.5889,  0.0000],
        [ 2.0000, -0.9092,  0.3120,  0.0000],
        [ 2.0000, -0.4612, -0.4900

In [26]:
for batch in tqdm(dl):
    data = batch[0].to(device)
    labels = batch[1].to(device)

 53%|█████▎    | 48172/90560 [02:04<01:49, 385.79it/s]


KeyboardInterrupt: 

In [29]:

! du -h ../data/01_processed/val_sets/cpg_pos.memmap/
! mkdir /tmp/cpg_pos.memmap
! mkdir /tmp/cpg_neg.memmap
! cd ../data/01_processed/val_sets/cpg_pos.memmap/ && time find  -type f -name '*.npy' | xargs -P16 -IX cp -v X $TMPDIR/cpg_pos.memmap/
! cd ../data/01_processed/val_sets/cpg_neg.memmap/ && time find  -type f -name '*.npy' | xargs -P16 -IX cp -v X $TMPDIR/cpg_neg.memmap/
! df -h ${TMPDIR:-/tmp}

12G	../data/01_processed/val_sets/cpg_pos.memmap/
mkdir: cannot create directory ‘/tmp/cpg_pos.memmap’: File exists
mkdir: cannot create directory ‘/tmp/cpg_neg.memmap’: File exists
'./shard_00004.npy' -> '/tmp/cpg_pos.memmap/shard_00004.npy'
'./shard_00003.npy' -> '/tmp/cpg_pos.memmap/shard_00003.npy'
'./shard_00002.npy' -> '/tmp/cpg_pos.memmap/shard_00002.npy'
'./shard_00005.npy' -> '/tmp/cpg_pos.memmap/shard_00005.npy'
'./shard_00010.npy' -> '/tmp/cpg_pos.memmap/shard_00010.npy'
'./shard_00008.npy' -> '/tmp/cpg_pos.memmap/shard_00008.npy'
'./shard_00006.npy' -> '/tmp/cpg_pos.memmap/shard_00006.npy'
'./shard_00001.npy' -> '/tmp/cpg_pos.memmap/shard_00001.npy'
'./shard_00011.npy' -> '/tmp/cpg_pos.memmap/shard_00011.npy'
'./shard_00000.npy' -> '/tmp/cpg_pos.memmap/shard_00000.npy'
'./shard_00007.npy' -> '/tmp/cpg_pos.memmap/shard_00007.npy'
'./shard_00009.npy' -> '/tmp/cpg_pos.memmap/shard_00009.npy'

real	0m0.798s
user	0m0.013s
sys	0m8.381s
'./shard_00002.npy' -> '/tmp/cpg_neg.memmap/

In [43]:
ds = LabeledMemmapDataset('/tmp/cpg_neg.memmap/', '/tmp/cpg_pos.memmap/', limit = 1_000_000)
dl = DataLoader(ds,
                batch_size=256,
                drop_last=True,
                persistent_workers=False,
                prefetch_factor=None,
                shuffle=True)

In [44]:
epoch_labels = [] 
for batch in tqdm(dl):
    data = batch[0]#.to(device)
    labels = batch[1]#.to(device)
    epoch_labels.append(labels)
    

100%|██████████| 3906/3906 [00:08<00:00, 487.09it/s]


In [45]:
total_labels = torch.stack(epoch_labels)
total_labels.sum()/total_labels.numel()

tensor(0.5000)

In [46]:
total_labels.shape

torch.Size([3906, 256])